目前已完成FullyConnectedNets的相关作业,暂时积累的调参经验做个小总结.(可能不太全面,也可能不太准确,但经验是慢慢积累的呀~)

目前的情况是网络搭好之后,对参数怎么调仍然不知所措....

### 超参数和初始值

需要调试的超参数:

- 学习率 the initial learning rate
- 学习率衰减 learning rate decay schedule (such as the decay constant)
- 正则化强度 regularization strength (L2 penalty, dropout strength)

需要调试的初始值:
- 权重的初始值 weight_scale

### 1. 在进行费时费力的最优化之前，最好进行一些合理性检查：

(1)**寻找特定情况的正确损失值。**在使用小参数进行初始化时，确保得到的损失值与期望一致。最好先单独检查数据损失（让正则化强度为0）。例如，对于一个跑CIFAR-10的Softmax分类器，一般期望它的初始损失值是2.302，这是因为初始时预计每个类别的概率是0.1（因为有10个类别），然后Softmax损失值正确分类的负对数概率：-ln(0.1)=2.302。对于Weston Watkins SVM，假设所有的边界都被越过（因为所有的分值都近似为零），所以损失值是9（因为对于每个错误分类，边界值是1）。如果没看到这些损失值，那么初始化中就可能有问题。

(2)**对小数据集过拟合:**在整个数据集进行训练之前，尝试在一个很小的数据集上进行训练（比如20个数据），然后确保能到达0的损失值。进行这个实验的时候，最好让正则化强度为0，不然它会阻止得到0的损失。除非能通过这一个正常性检查，不然进行整个数据集训练是没有意义的。但是注意，能对小数据集进行过拟合并不代表万事大吉，依然有可能存在不正确的实现。比如，因为某些错误，数据点的特征是随机的，这样算法也可能对小数据进行过拟合，但是在整个数据集上跑算法的时候，就没有任何泛化能力。

根据第二点来判断整个网络框架是否合适对吧?

### 2. 以周期为单位跟踪整个学习过程

#### 2.1 损失函数 : learning_rate , batch_size

<img src="https://pic4.zhimg.com/80/753f398b46cc28c1916d6703cf2080f5_hd.jpg">
跟踪损失函数主要是用来调节 learning_rate

左图展示了不同的学习率的效果。过低的学习率导致算法的改善是线性的。高一些的学习率会看起来呈几何指数下降，更高的学习率会让损失值很快下降，但是接着就停在一个不好的损失值上（绿线）。这是因为最优化的“能量”太大，参数在混沌中随机震荡，不能最优化到一个很好的点上。右图显示了一个典型的随时间变化的损失函数值，在CIFAR-10数据集上面训练了一个小的网络，这个损失函数值曲线看起来比较合理（虽然可能学习率有点小，但是很难说），而且指出了**批数据的数量可能有点太小（因为损失值的噪音很大）**。

损失值的震荡程度和批尺寸（batch size）有关，当批尺寸为1，震荡会相对较大。当批尺寸就是整个数据集时震荡就会最小，因为每个梯度更新都是单调地优化损失函数（除非学习率设置得过高）。

#### 2.2 训练集和验证集准确率: 正则化强度, 数据量

<img src="https://pic4.zhimg.com/80/05a6960a01c0204ced8d875ac3d91fba_hd.jpg">

蓝色的验证集曲线显示相较于训练集，验证集的准确率低了很多，这就说明模型有很强的过拟合。遇到这种情况，就应该增大正则化强度（更强的L2权重惩罚，更多的随机失活等）或收集更多的数据。另一种可能就是验证集曲线和训练集曲线如影随形，这种情况说明你的模型容量还不够大：应该通过增加参数数量让模型容量更大些。

#### 2.3 跟踪权重更新比例  learning_rate

最后一个应该跟踪的量是权重中更新值的数量和全部值的数量之间的比例。注意：是更新的，而不是原始梯度（比如，在普通sgd中就是梯度乘以学习率）。需要对每个参数集的更新比例进行单独的计算和跟踪。一个经验性的结论是这个比例应该在1e-3左右。如果更低，说明学习率可能太小，如果更高，说明学习率可能太高。下面是具体例子：
```python
#### 假设参数向量为W，其梯度向量为dW
param_scale = np.linalg.norm(W.ravel())
update = -learning_rate*dW # 简单SGD更新
update_scale = np.linalg.norm(update.ravel())
W += update # 实际更新
print(update_scale / param_scale) # 要得到1e-3左右
```
相较于跟踪最大和最小值，有研究者更喜欢计算和跟踪梯度的范式及其更新。这些矩阵通常是相关的，也能得到近似的结果。

In [13]:
import numpy as np
w = np.random.randn(2,3)
print(w.ravel())  ##array([-0.01629119,  0.02576669,  1.29815203, -0.37500137, -1.8310982 , -0.11184732])
param_scale = np.linalg.norm(w.ravel())   ## 默认是二范数
print(param_scale)
print(np.sqrt(np.sum(w*w)))

[-1.74320817  0.5778139  -0.18430115  0.13734742  0.7024064  -0.19925965]
1.98961152661
1.98961152661


#### 2.4 每层的激活数据及梯度分布

一个不正确的初始化可能让学习过程变慢，甚至彻底停止。还好，这个问题可以比较简单地诊断出来。其中一个方法是输出网络中所有层的激活数据和梯度分布的柱状图。直观地说，就是如果看到任何奇怪的分布情况，那都不是好兆头。比如，对于使用tanh的神经元，我们应该看到激活数据的值在整个[-1,1]区间中都有分布。如果看到神经元的输出全部是0，或者全都饱和了往-1和1上跑，那肯定就是有问题了。

### 第一层可视化

最后，如果数据是图像像素数据，那么把第一层特征可视化会有帮助. 第一层 $W1.shape=(3072,H_1)$

前面说了这么多,都是来判断超参数的好坏的,但具体操作起来又是如何的呢?

### 3. 超参数调试

**3.1 超参数范围。在对数尺度上进行超参数搜索。**例如，一个典型的学习率应该看起来是这样：```learning_rate = 10**uniform(-6, 1)```。也就是说，我们从标准分布中随机生成了一个数字，然后让它成为10的阶数。对于正则化强度，可以采用同样的策略。直观地说，这是因为学习率和正则化强度都对于训练的动态进程有乘的效果。例如：当学习率是0.001的时候，如果对其固定地增加0.01，那么对于学习进程会有很大影响。然而当学习率是10的时候，影响就微乎其微了。这就是因为学习率乘以了计算出的梯度。因此，**比起加上或者减少某些值，思考学习率的范围是乘以或者除以某些值更加自然**。但是有一些参数（比如随机失活）还是在原始尺度上进行搜索（例如：dropout=uniform(0,1)）。

#### 3.2 bibysitting
实现。更大的神经网络需要更长的时间去训练，所以调参可能需要几天甚至几周。记住这一点很重要，因为这会影响你设计代码的思路。一个具体的设计是用仆程序持续地随机设置参数然后进行最优化。在训练过程中，仆程序会对每个周期后验证集的准确率进行监控，然后向文件系统写下一个模型的记录点（记录点中有各种各样的训练统计数据，比如随着时间的损失值变化等），这个文件系统最好是可共享的。在文件名中最好包含验证集的算法表现，这样就能方便地查找和排序了。然后还有一个主程序，它可以启动或者结束计算集群中的仆程序，有时候也可能根据条件查看仆程序写下的记录点，输出它们的训练统计数据等。

#### 3.3 随机搜索优于网格搜索。
Bergstra和Bengio在文章[Random Search for Hyper-Parameter Optimization](http://link.zhihu.com/?target=http%3A//www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)中说“随机选择比网格化的选择更加有效”，而且在实践中也更容易实现。
<img src="https://pic1.zhimg.com/80/d25cf561835c7b96ae6d1c91868bcbff_hd.jpg">

在Random Search for Hyper-Parameter Optimization中的核心说明图。通常，有些超参数比其余的更重要，通过随机搜索，而不是网格化的搜索，可以让你更精确地发现那些比较重要的超参数的好数值。

#### 对于边界上的最优值要小心。
这种情况一般发生在你在一个不好的范围内搜索超参数（比如学习率）的时候。比如，假设我们使用```learning_rate = 10 ** uniform(-6,1)```来进行搜索。一旦我们得到一个比较好的值，一定要确认你的值不是出于这个范围的边界上，不然你可能错过更好的其他搜索范围。

#### 从粗到细地分阶段搜索。
在实践中，先进行初略范围（比如```10 ** [-6, 1]```）搜索，然后根据好的结果出现的地方，缩小范围进行搜索。进行粗搜索的时候，让模型训练一个周期就可以了，因为很多超参数的设定会让模型没法学习，或者突然就爆出很大的损失值。第二个阶段就是对一个更小的范围进行搜索，这时可以让模型运行5个周期，而最后一个阶段就在最终的范围内进行仔细搜索，运行很多次周期。

#### 贝叶斯超参数最优化是一整个研究领域，
主要是研究在超参数空间中更高效的导航算法。其核心的思路是在不同超参数设置下查看算法性能时，要在探索和使用中进行合理的权衡。基于这些模型，发展出很多的库，比较有名的有： Spearmint, SMAC, 和Hyperopt。然而，在卷积神经网络的实际使用中，比起上面介绍的先认真挑选的一个范围，然后在该范围内随机搜索的方法，这个方法还是差一些。这里有更详细的讨论。

### 4. 模型集成

在实践的时候，有一个总是能提升神经网络几个百分点准确率的办法，就是在训练的时候训练几个独立的模型，然后在测试的时候平均它们预测结果。集成的模型数量增加，算法的结果也单调提升（但提升效果越来越少）。还有模型之间的差异度越大，提升效果可能越好。进行集成有以下几种方法：

#### 4.1 同一个模型，不同的初始化。
使用交叉验证来得到最好的超参数，然后用最好的参数来训练不同初始化条件的模型。这种方法的风险在于多样性只来自于不同的初始化条件。

#### 4.2 在交叉验证中发现最好的模型。
使用交叉验证来得到最好的超参数，然后取其中最好的几个（比如10个）模型来进行集成。这样就提高了集成的多样性，但风险在于可能会包含不够理想的模型。在实际操作中，这样操作起来比较简单，在交叉验证后就不需要额外的训练了。

#### 4.3 一个模型设置多个记录点。
如果训练非常耗时，那就在不同的训练时间对网络留下记录点（比如每个周期结束），然后用它们来进行模型集成。很显然，这样做多样性不足，但是在实践中效果还是不错的，这种方法的优势是代价比较小。

#### 4.4 在训练的时候跑参数的平均值。
和上面一点相关的，还有一个也能得到1-2个百分点的提升的小代价方法，这个方法就是在训练过程中，如果损失值相较于前一次权重出现指数下降时，就在内存中对网络的权重进行一个备份。这样你就对前几次循环中的网络状态进行了平均。你会发现这个“平滑”过的版本的权重总是能得到更少的误差。直观的理解就是目标函数是一个碗状的，你的网络在这个周围跳跃，所以对它们平均一下，就更可能跳到中心去。

模型集成的一个劣势就是在测试数据的时候会花费更多时间。最近Geoff Hinton在“Dark Knowledge”上的工作很有启发：其思路是通过将集成似然估计纳入到修改的目标函数中，从一个好的集成中抽出一个单独模型。